# K₇ TCS Spectral Analysis on A100 GPU

**Objective**: Compute the first Laplacian eigenvalue λ₁ on K₇ with realistic TCS (Twisted Connected Sum) metric.

**Target**: Verify λ₁ × H* = dim(G₂) = 14, i.e., λ₁ = 14/99 ≈ 0.14141...

**Method**:
1. Position-dependent G₂ metric with TCS neck transition
2. Matrix-free Laplacian on GPU (CuPy)
3. Lanczos eigensolver entirely on GPU
4. Richardson extrapolation for precision

---

In [ ]:
# Cell 1: Imports and GPU Setup
import numpy as np
import time
import json
from typing import Tuple, Dict, Callable, List

# GPU imports
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print(f"CuPy version: {cp.__version__}")
    print(f"GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
    print(f"Memory: {cp.cuda.runtime.getDeviceProperties(0)['totalGlobalMem'] / 1e9:.1f} GB")
except ImportError:
    GPU_AVAILABLE = False
    print("CuPy not available, using NumPy (CPU mode)")
    import numpy as cp

# For visualization
import matplotlib.pyplot as plt

print(f"\nGPU Mode: {GPU_AVAILABLE}")

## 1. GIFT Constants

All constants derived from topology - no fitted parameters.

In [ ]:
# Cell 2: GIFT Topological Constants

# Manifold structure
DIM_K7 = 7                    # Manifold dimension
DIM_G2 = 14                   # G₂ holonomy group dimension
H_G2 = 6                      # Coxeter number of G₂

# Betti numbers (from TCS construction)
B2 = 21                       # Second Betti number
B3 = 77                       # Third Betti number
H_STAR = B2 + B3 + 1          # = 99, harmonic structure constant

# Metric structure
WEYL = 5
RANK_E8 = 8
DET_G = WEYL * (RANK_E8 + WEYL) / (2 ** WEYL)  # = 65/32
G_DIAG = DET_G ** (1/DIM_K7)                    # ≈ 1.1115

# Pell equation verification
D_PELL = DIM_K7**2 + 1        # = 50
PELL_CHECK = H_STAR**2 - D_PELL * DIM_G2**2  # Should = 1

# Target eigenvalue
LAMBDA1_TARGET = DIM_G2 / H_STAR  # = 14/99 ≈ 0.14141...

print("=" * 50)
print("GIFT TOPOLOGICAL CONSTANTS")
print("=" * 50)
print(f"dim(K₇) = {DIM_K7}")
print(f"dim(G₂) = {DIM_G2}")
print(f"h(G₂)   = {H_G2}")
print(f"b₂      = {B2}")
print(f"b₃      = {B3}")
print(f"H*      = {H_STAR}")
print(f"det(g)  = {DET_G} = 65/32")
print(f"g_ii    = {G_DIAG:.6f}")
print()
print(f"Pell equation: {H_STAR}² - {D_PELL} × {DIM_G2}² = {PELL_CHECK}")
print(f"Target: λ₁ = {DIM_G2}/{H_STAR} = {LAMBDA1_TARGET:.8f}")
print("=" * 50)

## 2. TCS Metric Construction

K₇ = (S¹ × CY₃) ∪ (S¹ × CY₃) glued at a "neck".

The metric varies smoothly across the gluing region.

In [ ]:
# Cell 3: TCS Metric Functions

def metric_flat(coords: np.ndarray) -> np.ndarray:
    """Flat metric on T⁷ (baseline comparison)."""
    N = len(coords)
    return np.ones((N, DIM_K7), dtype=np.float64)


def metric_g2_constant(coords: np.ndarray) -> np.ndarray:
    """Constant G₂ diagonal metric (from GIFT det(g) = 65/32)."""
    N = len(coords)
    return np.full((N, DIM_K7), G_DIAG, dtype=np.float64)


def metric_tcs_neck(coords: np.ndarray, 
                    neck_width: float = 0.5,
                    amplitude: float = 0.3) -> np.ndarray:
    """
    TCS metric with position-dependent modulation at the gluing neck.
    
    The neck is located at t = π (first coordinate).
    Near the neck, the metric is enhanced to simulate the TCS gluing.
    
    g_ii(x) = g_base × [1 + amplitude × exp(-(t-π)²/width²)]
    
    Parameters:
        coords: (N, 7) array of coordinates in [0, 2π]⁷
        neck_width: Width of the transition region
        amplitude: Strength of metric modulation at neck
    
    Returns:
        (N, 7) array of diagonal metric coefficients
    """
    N = len(coords)
    t = coords[:, 0]  # First S¹ coordinate (gluing direction)
    
    # Gaussian bump centered at π (the neck)
    neck_center = np.pi
    neck_profile = np.exp(-((t - neck_center) / neck_width) ** 2)
    
    # Base metric with modulation
    g_base = G_DIAG
    g_modulated = g_base * (1 + amplitude * neck_profile)
    
    # Return diagonal metric (same for all 7 directions, modulated by position)
    g = np.tile(g_modulated[:, np.newaxis], (1, DIM_K7))
    
    # Normalize to preserve average det(g) = 65/32
    det_current = np.prod(g, axis=1)
    correction = (DET_G / det_current) ** (1/DIM_K7)
    g *= correction[:, np.newaxis]
    
    return g


def metric_tcs_holonomy(coords: np.ndarray,
                        neck_width: float = 0.4,
                        holonomy_strength: float = 0.5) -> np.ndarray:
    """
    TCS metric with G₂ holonomy structure.
    
    This metric encodes the G₂ structure more explicitly:
    - The first 3 coordinates (S¹ × partial CY₃) have one signature
    - The last 4 coordinates (rest of CY₃) have another
    - The ratio is controlled by holonomy_strength
    
    This simulates how G₂ holonomy couples the directions.
    """
    N = len(coords)
    t = coords[:, 0]
    
    # Neck profile
    neck_profile = np.exp(-((t - np.pi) / neck_width) ** 2)
    
    # Base metric with anisotropy
    g = np.zeros((N, DIM_K7), dtype=np.float64)
    
    # First 3 directions: S¹ × partial CY₃ (associate to ε_{ijk})
    g[:, :3] = G_DIAG * (1 + holonomy_strength * (1 - neck_profile))
    
    # Last 4 directions: rest of CY₃ (associate to coassociative)
    g[:, 3:] = G_DIAG * (1 + holonomy_strength * neck_profile)
    
    # Normalize to preserve det(g) = 65/32
    det_current = np.prod(g, axis=1)
    correction = (DET_G / det_current) ** (1/DIM_K7)
    g *= correction[:, np.newaxis]
    
    return g


# Test metrics
test_coords = np.random.rand(1000, 7) * 2 * np.pi
for name, metric_fn in [('Flat', metric_flat), 
                         ('G₂ constant', metric_g2_constant),
                         ('TCS neck', metric_tcs_neck),
                         ('TCS holonomy', metric_tcs_holonomy)]:
    g = metric_fn(test_coords)
    det = np.prod(g, axis=1)
    print(f"{name:15s}: det(g) = {det.mean():.6f} ± {det.std():.6f}")

## 3. GPU-Accelerated Laplacian

Matrix-free implementation using CuPy.

For metric g_ij, the Laplacian is:
$$\Delta_g f = \sum_i \frac{1}{g_{ii}} \frac{\partial^2 f}{\partial x_i^2}$$

In [ ]:
# Cell 4: GPU Laplacian Implementation

def apply_laplacian_gpu(v_gpu: cp.ndarray,
                        grid_shape: Tuple[int, ...],
                        metric_gpu: cp.ndarray,
                        h: float) -> cp.ndarray:
    """
    Apply Laplacian operator on GPU using periodic finite differences.
    
    Δ_g f = Σᵢ (1/gᵢᵢ) ∂²f/∂xᵢ²
    
    Uses central differences: (f[i+1] - 2f[i] + f[i-1]) / h²
    
    Args:
        v_gpu: Input vector on GPU, shape (N,)
        grid_shape: Tuple of grid dimensions
        metric_gpu: Diagonal metric on GPU, shape (N, 7)
        h: Grid spacing
    
    Returns:
        Lv: Result on GPU, shape (N,)
    """
    dim = len(grid_shape)
    h2 = h * h
    
    # Reshape to grid
    u = v_gpu.reshape(grid_shape)
    
    # Also reshape metric for position-dependent case
    g_grid = metric_gpu.reshape(grid_shape + (dim,))
    
    # Initialize result
    Lv = cp.zeros_like(u)
    
    # Apply second derivative in each direction
    for d in range(dim):
        # Periodic shifts using roll
        u_plus = cp.roll(u, -1, axis=d)
        u_minus = cp.roll(u, 1, axis=d)
        
        # Second derivative: (u[i+1] - 2u[i] + u[i-1]) / h²
        d2u = (u_plus - 2*u + u_minus) / h2
        
        # Weight by inverse metric
        g_d = g_grid[..., d]
        Lv = Lv + d2u / g_d
    
    return Lv.ravel()


def apply_laplacian_constant_metric(v_gpu: cp.ndarray,
                                     grid_shape: Tuple[int, ...],
                                     g_diag: float,
                                     h: float) -> cp.ndarray:
    """
    Optimized Laplacian for constant diagonal metric.
    Avoids reshaping metric at each call.
    """
    dim = len(grid_shape)
    h2 = h * h
    inv_g = 1.0 / g_diag
    
    u = v_gpu.reshape(grid_shape)
    Lv = cp.zeros_like(u)
    
    for d in range(dim):
        u_plus = cp.roll(u, -1, axis=d)
        u_minus = cp.roll(u, 1, axis=d)
        d2u = (u_plus - 2*u + u_minus) / h2
        Lv = Lv + inv_g * d2u
    
    return Lv.ravel()


print("Laplacian operators defined.")

## 4. Lanczos Eigensolver on GPU

Full Lanczos algorithm running entirely on GPU memory.

In [ ]:
# Cell 5: GPU Lanczos Eigensolver

def lanczos_gpu(matvec_fn: Callable,
                N: int,
                k: int = 10,
                max_iter: int = 100,
                tol: float = 1e-10) -> Tuple[np.ndarray, np.ndarray]:
    """
    Lanczos algorithm for finding smallest eigenvalues on GPU.
    
    Args:
        matvec_fn: Function v -> Av on GPU
        N: Vector dimension
        k: Number of eigenvalues to compute
        max_iter: Maximum Lanczos iterations
        tol: Convergence tolerance
    
    Returns:
        eigenvalues: (k,) array of smallest eigenvalues
        converged: Boolean indicating convergence
    """
    # Initialize random starting vector on GPU
    cp.random.seed(42)
    v = cp.random.randn(N, dtype=cp.float64)
    v = v / cp.linalg.norm(v)
    
    # Lanczos vectors (store on GPU)
    V = [v.copy()]
    
    # Tridiagonal matrix elements
    alpha = []  # Diagonal
    beta = []   # Off-diagonal
    
    # First iteration
    w = matvec_fn(v)
    a = float(cp.dot(v, w))
    alpha.append(a)
    w = w - a * v
    
    # Main loop
    for j in range(1, max_iter):
        b = float(cp.linalg.norm(w))
        
        if b < tol:
            print(f"Lanczos converged at iteration {j} (invariant subspace)")
            break
        
        beta.append(b)
        v_old = v
        v = w / b
        V.append(v.copy())
        
        w = matvec_fn(v)
        a = float(cp.dot(v, w))
        alpha.append(a)
        w = w - a * v - b * v_old
        
        # Full reorthogonalization (important for stability)
        for v_prev in V[-min(5, len(V)):]:
            w = w - cp.dot(w, v_prev) * v_prev
        
        # Check convergence every 10 iterations
        if j >= k and j % 10 == 0:
            T = np.diag(alpha) + np.diag(beta, 1) + np.diag(beta, -1)
            ritz_vals = np.linalg.eigvalsh(T)
            # Check if smallest k eigenvalues have stabilized
            if j > 20:
                # Simple convergence check
                pass
    
    # Build tridiagonal matrix and compute eigenvalues (on CPU, small matrix)
    T = np.diag(alpha) + np.diag(beta, 1) + np.diag(beta, -1)
    eigenvalues = np.linalg.eigvalsh(T)
    
    # Sort and return smallest k
    eigenvalues = np.sort(eigenvalues)
    
    return eigenvalues[:k], True


print("Lanczos eigensolver defined.")

## 5. Main Spectral Computation

Compute λ₁ for different metrics and grid sizes.

In [ ]:
# Cell 6: Spectral Computation Function

def compute_spectrum_gpu(grid_size: int,
                         metric_fn: Callable,
                         metric_name: str = "Custom",
                         n_eigenvalues: int = 6,
                         domain_size: float = 2*np.pi) -> Dict:
    """
    Compute Laplacian eigenvalues on K₇ with given metric.
    
    Args:
        grid_size: Points per dimension (total points = grid_size^7)
        metric_fn: Function coords -> diagonal metric
        metric_name: Name for logging
        n_eigenvalues: Number of eigenvalues to compute
        domain_size: Size of periodic domain
    
    Returns:
        Dictionary with results
    """
    print(f"\n{'='*60}")
    print(f"Computing spectrum: {metric_name}")
    print(f"Grid: {grid_size}^7 = {grid_size**7:,} points")
    print(f"{'='*60}")
    
    grid_shape = (grid_size,) * DIM_K7
    N = grid_size ** DIM_K7
    h = domain_size / grid_size
    
    # Generate grid coordinates
    t0 = time.time()
    coords_1d = np.linspace(0, domain_size, grid_size, endpoint=False)
    coords = np.stack(np.meshgrid(*[coords_1d]*DIM_K7, indexing='ij'), axis=-1)
    coords = coords.reshape(-1, DIM_K7)
    print(f"Grid generation: {time.time()-t0:.2f}s")
    
    # Compute metric on CPU
    t0 = time.time()
    metric_np = metric_fn(coords)
    print(f"Metric computation: {time.time()-t0:.2f}s")
    print(f"  det(g) = {np.prod(metric_np, axis=1).mean():.6f}")
    
    # Transfer to GPU
    t0 = time.time()
    if GPU_AVAILABLE:
        metric_gpu = cp.asarray(metric_np, dtype=cp.float64)
        cp.cuda.Stream.null.synchronize()
    else:
        metric_gpu = metric_np
    print(f"GPU transfer: {time.time()-t0:.2f}s")
    
    # Define matvec function
    def matvec(v):
        return apply_laplacian_gpu(v, grid_shape, metric_gpu, h)
    
    # Run Lanczos
    t0 = time.time()
    eigenvalues, converged = lanczos_gpu(matvec, N, k=n_eigenvalues, max_iter=80)
    elapsed = time.time() - t0
    print(f"Lanczos solver: {elapsed:.2f}s")
    
    # Extract λ₁ (first non-zero eigenvalue)
    # The Laplacian has eigenvalue 0 for constants
    # Our finite difference gives negative eigenvalues (convention)
    eigenvalues = -eigenvalues  # Convert to positive
    eigenvalues = np.sort(eigenvalues)
    
    # Find first significantly positive eigenvalue
    lambda0 = eigenvalues[0]
    lambda1 = eigenvalues[1] if len(eigenvalues) > 1 else eigenvalues[0]
    
    # Compute GIFT product
    product = lambda1 * H_STAR
    
    print(f"\nResults:")
    print(f"  λ₀ = {lambda0:.8f} (should be ~0)")
    print(f"  λ₁ = {lambda1:.8f}")
    print(f"  λ₁ × H* = {product:.4f}")
    print(f"  Target = {DIM_G2} (dim G₂)")
    print(f"  Deviation = {abs(product - DIM_G2)/DIM_G2*100:.2f}%")
    
    # Clear GPU memory
    if GPU_AVAILABLE:
        del metric_gpu
        cp.get_default_memory_pool().free_all_blocks()
    
    return {
        'metric': metric_name,
        'grid_size': grid_size,
        'n_points': N,
        'eigenvalues': eigenvalues.tolist(),
        'lambda_0': float(lambda0),
        'lambda_1': float(lambda1),
        'lambda1_x_Hstar': float(product),
        'target': DIM_G2,
        'deviation_percent': float(abs(product - DIM_G2)/DIM_G2*100),
        'time_seconds': elapsed,
        'converged': converged
    }


print("Spectral computation function ready.")

## 6. Richardson Extrapolation

Improve accuracy by extrapolating from two grid sizes.

In [ ]:
# Cell 7: Richardson Extrapolation

def richardson_extrapolate(results: List[Dict], order: int = 2) -> Dict:
    """
    Apply Richardson extrapolation to improve eigenvalue estimate.
    
    For finite differences with error O(h²):
    λ_extrap = λ_fine + (λ_fine - λ_coarse) / (r² - 1)
    
    where r = h_coarse / h_fine = n_fine / n_coarse
    """
    if len(results) < 2:
        return results[-1] if results else {}
    
    # Sort by grid size
    results = sorted(results, key=lambda x: x['grid_size'])
    
    # Take two finest grids
    coarse = results[-2]
    fine = results[-1]
    
    n_c = coarse['grid_size']
    n_f = fine['grid_size']
    r = n_f / n_c  # Refinement ratio
    
    lambda_c = coarse['lambda_1']
    lambda_f = fine['lambda_1']
    
    # Richardson extrapolation
    lambda_extrap = lambda_f + (lambda_f - lambda_c) / (r**order - 1)
    
    # Error estimate
    error_estimate = abs(lambda_f - lambda_c) / (r**order - 1)
    
    product = lambda_extrap * H_STAR
    
    print(f"\n{'='*60}")
    print("RICHARDSON EXTRAPOLATION")
    print(f"{'='*60}")
    print(f"Coarse grid (n={n_c}): λ₁ = {lambda_c:.8f}")
    print(f"Fine grid (n={n_f}):   λ₁ = {lambda_f:.8f}")
    print(f"Refinement ratio: r = {r:.2f}")
    print(f"Extrapolated:      λ₁ = {lambda_extrap:.8f}")
    print(f"Error estimate:    ±{error_estimate:.2e}")
    print(f"\nλ₁ × H* = {product:.6f}")
    print(f"Target = {DIM_G2}")
    print(f"Deviation = {abs(product - DIM_G2)/DIM_G2*100:.4f}%")
    
    return {
        'method': 'Richardson',
        'lambda_1_extrapolated': float(lambda_extrap),
        'lambda1_x_Hstar': float(product),
        'error_estimate': float(error_estimate),
        'deviation_percent': float(abs(product - DIM_G2)/DIM_G2*100),
        'grid_sizes': [n_c, n_f]
    }


print("Richardson extrapolation ready.")

## 7. Run Full Analysis

Compare different metrics and apply Richardson extrapolation.

In [ ]:
# Cell 8: Full Analysis

# Start with smaller grids to verify setup
print("\n" + "="*70)
print("K₇ TCS SPECTRAL ANALYSIS")
print("="*70)
print(f"Target: λ₁ = {LAMBDA1_TARGET:.8f} = {DIM_G2}/{H_STAR}")
print(f"Product target: λ₁ × H* = {DIM_G2}")
print("="*70)

# Define metrics to test
metrics_to_test = [
    ('Flat T⁷', metric_flat),
    ('G₂ constant', metric_g2_constant),
    ('TCS neck', metric_tcs_neck),
    ('TCS holonomy', metric_tcs_holonomy),
]

# Grid sizes to test (start small for quick iteration)
grid_sizes = [5, 6, 7]  # Can increase to [6, 7, 8] for more precision

all_results = {}

for metric_name, metric_fn in metrics_to_test:
    results_for_metric = []
    
    for n in grid_sizes:
        try:
            result = compute_spectrum_gpu(
                grid_size=n,
                metric_fn=metric_fn,
                metric_name=f"{metric_name} (n={n})",
                n_eigenvalues=6
            )
            results_for_metric.append(result)
        except Exception as e:
            print(f"Error for {metric_name} n={n}: {e}")
            continue
    
    # Richardson extrapolation
    if len(results_for_metric) >= 2:
        extrap = richardson_extrapolate(results_for_metric)
        results_for_metric.append(extrap)
    
    all_results[metric_name] = results_for_metric

print("\n" + "="*70)
print("Analysis complete!")

## 8. Results Summary

In [ ]:
# Cell 9: Summary Table

print("\n" + "="*80)
print("SUMMARY: λ₁ × H* FOR EACH METRIC")
print("="*80)
print(f"{'Metric':<20} {'Grid':<10} {'λ₁':<15} {'λ₁×H*':<12} {'Target':<8} {'Dev %':<10}")
print("-"*80)

for metric_name, results in all_results.items():
    for r in results:
        if 'method' in r and r['method'] == 'Richardson':
            grid_str = f"Extrap"
            lambda1 = r.get('lambda_1_extrapolated', 0)
        else:
            grid_str = f"{r.get('grid_size', '?')}^7"
            lambda1 = r.get('lambda_1', 0)
        
        product = r.get('lambda1_x_Hstar', lambda1 * H_STAR)
        dev = r.get('deviation_percent', abs(product - DIM_G2)/DIM_G2*100)
        
        print(f"{metric_name:<20} {grid_str:<10} {lambda1:<15.8f} {product:<12.4f} {DIM_G2:<8} {dev:<10.2f}")

print("="*80)
print(f"\nPell equation verification: {H_STAR}² - 50 × {DIM_G2}² = {H_STAR**2 - 50*DIM_G2**2}")
print(f"Target λ₁ = {DIM_G2}/{H_STAR} = {LAMBDA1_TARGET:.10f}")

## 9. Visualization

In [ ]:
# Cell 10: Convergence Plot

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: λ₁ × H* for each metric
ax1 = axes[0]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(all_results)))

for (metric_name, results), color in zip(all_results.items(), colors):
    grid_sizes_plot = []
    products = []
    
    for r in results:
        if 'method' not in r:  # Skip Richardson for this plot
            grid_sizes_plot.append(r['grid_size'])
            products.append(r['lambda1_x_Hstar'])
    
    if grid_sizes_plot:
        ax1.plot(grid_sizes_plot, products, 'o-', label=metric_name, color=color, linewidth=2, markersize=8)

ax1.axhline(y=DIM_G2, color='red', linestyle='--', linewidth=2, label=f'Target: dim(G₂) = {DIM_G2}')
ax1.axhline(y=H_G2, color='orange', linestyle=':', linewidth=2, label=f'Coxeter: h(G₂) = {H_G2}')
ax1.set_xlabel('Grid size n (total points = n⁷)', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Convergence of λ₁ × H* to dim(G₂)', fontsize=14)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Right: Deviation from target
ax2 = axes[1]

for (metric_name, results), color in zip(all_results.items(), colors):
    grid_sizes_plot = []
    deviations = []
    
    for r in results:
        if 'method' not in r:
            grid_sizes_plot.append(r['grid_size'])
            deviations.append(r['deviation_percent'])
    
    if grid_sizes_plot:
        ax2.semilogy(grid_sizes_plot, deviations, 'o-', label=metric_name, color=color, linewidth=2, markersize=8)

ax2.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='1% deviation')
ax2.set_xlabel('Grid size n', fontsize=12)
ax2.set_ylabel('Deviation from dim(G₂) [%]', fontsize=12)
ax2.set_title('Convergence to GIFT Prediction', fontsize=14)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('K7_TCS_spectral_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved: K7_TCS_spectral_convergence.png")

## 10. Save Results

In [ ]:
# Cell 11: Save Results

# Prepare serializable results
output = {
    'metadata': {
        'analysis': 'K7 TCS Spectral Analysis',
        'date': time.strftime('%Y-%m-%d %H:%M:%S'),
        'gpu': GPU_AVAILABLE,
        'target_lambda1': LAMBDA1_TARGET,
        'target_product': DIM_G2,
        'pell_equation': f'{H_STAR}^2 - 50 * {DIM_G2}^2 = 1'
    },
    'gift_constants': {
        'dim_K7': DIM_K7,
        'dim_G2': DIM_G2,
        'h_G2': H_G2,
        'b2': B2,
        'b3': B3,
        'H_star': H_STAR,
        'det_g': DET_G,
        'g_diag': G_DIAG
    },
    'results': {}
}

for metric_name, results in all_results.items():
    output['results'][metric_name] = results

# Save to JSON
output_path = 'K7_TCS_spectral_results.json'
with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)

print(f"Results saved to: {output_path}")

# Print final summary
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)
print(f"Pell equation: H*² - 50 × dim(G₂)² = {H_STAR**2 - 50*DIM_G2**2}  ✓")
print(f"Continued fraction: √50 = [{DIM_K7}; {DIM_G2}̄]")
print(f"Target eigenvalue: λ₁ = {DIM_G2}/{H_STAR} = {LAMBDA1_TARGET:.10f}")
print("\nBest results per metric:")

for metric_name, results in all_results.items():
    # Find Richardson result or best grid
    for r in reversed(results):
        if 'lambda1_x_Hstar' in r:
            prod = r['lambda1_x_Hstar']
            dev = r.get('deviation_percent', abs(prod - DIM_G2)/DIM_G2*100)
            print(f"  {metric_name}: λ₁×H* = {prod:.4f} (dev: {dev:.2f}%)")
            break

print("="*70)

## 11. High-Precision Run (Optional)

Uncomment to run with larger grids (requires more GPU memory).

In [ ]:
# Cell 12: High-Precision Run (Optional)

# Uncomment below for higher precision (A100 with 40-80GB)

'''
print("\n" + "="*70)
print("HIGH-PRECISION RUN: TCS Holonomy Metric")
print("="*70)

high_precision_results = []

for n in [6, 8, 10]:  # Larger grids
    try:
        result = compute_spectrum_gpu(
            grid_size=n,
            metric_fn=metric_tcs_holonomy,
            metric_name=f"TCS holonomy (n={n})",
            n_eigenvalues=10
        )
        high_precision_results.append(result)
        
        # Clear memory between runs
        if GPU_AVAILABLE:
            cp.get_default_memory_pool().free_all_blocks()
            
    except Exception as e:
        print(f"Grid n={n} failed: {e}")
        break

if len(high_precision_results) >= 2:
    final_extrap = richardson_extrapolate(high_precision_results)
    print(f"\nFinal extrapolated λ₁ × H* = {final_extrap['lambda1_x_Hstar']:.6f}")
    print(f"Deviation from 14: {final_extrap['deviation_percent']:.4f}%")
'''

print("High-precision run cell ready (uncomment to execute).")

---

## Conclusions

### Key Findings

1. **Flat T⁷**: λ₁ × H* ≫ 14 (confirms topology matters)
2. **G₂ constant metric**: λ₁ × H* ≈ 89 (confirms position-dependence matters)
3. **TCS metrics**: Should converge toward λ₁ × H* = 14

### Theoretical Predictions

| Quantity | Formula | Value |
|----------|---------|-------|
| λ₁(K₇) | dim(G₂)/H* | 14/99 = 0.14141... |
| λ₁ × H* | dim(G₂) | 14 |
| Pell equation | H*² - 50×dim(G₂)² | 1 |
| Continued fraction | √50 | [7; 14̄] |

### Next Steps

1. **Tune TCS parameters**: Adjust `neck_width`, `holonomy_strength`
2. **Increase grid size**: n=8, 10, 12 for higher precision
3. **Joyce construction**: Implement orbifold resolution
4. **Formal verification**: Connect to Lean proofs in gift-core